In [ ]:
from __future__ import print_function
import copy
from threading import Condition, Thread
import numpy as np
import numpy.random as npr
import torch as th
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.modules.loss import CrossEntropyLoss, MSELoss
import torch.nn.functional as F
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader, TensorDataset
import my

In [ ]:
import matplotlib.pylab as pl
%matplotlib inline

In [ ]:
N_TRAIN, N_TEST = 0, 0
BATCH_SIZE = 64
cuda = True

# train_data, train_labels, test_data, test_labels = my.unbalanced_cifar10(N_TRAIN, N_TEST, p=[0, 1, 10])
train_data, train_labels, test_data, test_labels = my.unbalanced_cifar10(N_TRAIN, N_TEST, p=[])

train_data_np, train_labels_np, test_data_np, test_labels_np = \
    train_data, train_labels, test_data, test_labels
    
train_data = th.from_numpy(train_data).float()
train_labels = th.from_numpy(train_labels).long()
test_data = th.from_numpy(test_data).float()
test_labels = th.from_numpy(test_labels).long()

if cuda:
    th.cuda.set_device(2)

N_FEATURES = train_data.size()[1]
N_CLASSES = int(train_labels.max() - train_labels.min() + 1)

In [ ]:
class CNN(nn.Module):
    def __init__(self, n_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 2, 1)
        self.conv2 = nn.Conv2d(16, 8, 3, 2, 1)
        self.linear = nn.Linear(8, n_classes)
    
    def forward(self, x):
        if x.dim() != 4:
            x = x.view(-1, 3, 32, 32)
        x = F.tanh(self.conv1(x))
        x = F.tanh(self.conv2(x))
        x = F.avg_pool2d(x, 8)
        x = self.linear(x.view(-1, 8))
        return x

In [ ]:
SAMPLE_SIZE = 64

th.random.manual_seed(1)
th.cuda.manual_seed_all(1)

for x, y in DataLoader(TensorDataset(train_data, train_labels), SAMPLE_SIZE, shuffle=True):
    if cuda:
        x, y = x.cuda(), y.cuda()
    x, y = Variable(x), Variable(y)
    break

L = lambda y_bar: my.nd_f_beta(th.max(y_bar, 1)[1], y, N_CLASSES)

def perturb_y(y, std):
    return y + (lambda x: x.cuda() if cuda else x)(Variable(th.randn(y.size()) * std))

def critic_forward(y_bar, detach=False):
    z_bar = F.softmax(y_bar, 1)
    z = my.onehot(y, N_CLASSES)
    x = th.cat((z_bar, z), 1).view(1, -1)
    return critic((lambda x: x.detach() if detach else x)(x))

def critic_mse(y_bar, y_perturbed, target):
    l_bar = critic_forward(y_bar, True)
    l_perturbed = critic_forward(y_perturbed, True)
#     return th.mean(th.exp(target ** 2 / 0.1) * ((l_perturbed - l_bar) - target) ** 2)
    return th.mean(((l_perturbed - l_bar) - target) ** 2)

In [ ]:
th.random.manual_seed(1)
th.cuda.manual_seed_all(1)

# c = my.MLP((N_FEATURES,) + (512,) * 1 + (N_CLASSES,), F.relu)
c = CNN(N_CLASSES)
critic = my.RN(SAMPLE_SIZE, 2 * N_CLASSES, (512,) * 3 + (1,), F.relu)

if cuda:
    c.cuda()
    critic.cuda()

# c_optim = SGD(c.parameters(), 0.1, momentum=0.5)
# critic_optim = SGD(critic.parameters(), 0.1, momentum=0.5)
c_optim = Adam(c.parameters(), 1e-3)
critic_optim = Adam(critic.parameters(), 1e-3)

float(L(c(x)))

In [ ]:
N_ITERATIONS = 250
N_PERTURBATIONS = 25
STD = 1e-0
CRITIC_N_ITERATIONS = 10
CLASSIFIER_N_ITERATIONS = 25

# TODO replay buffer
hist = []
for i in range(N_ITERATIONS):
    hist.append({})
#     hist[-1]['c_state_dict'] = copy.deepcopy(c.state_dict())

    c.eval()
    critic.train()
    y_bar = c(x)
    L_bar = L(y_bar)
    l_bar = critic_forward(y_bar, detach=True)

    y_list, target_list, w_list = [], [], []
    for j in range(N_PERTURBATIONS):
        y_perturbed = perturb_y(y_bar, STD) # TODO perturb in simplex
        L_perturbed = L(y_perturbed)
        target = L_perturbed - L_bar
        y_list.append(y_perturbed)
        target_list.append(target)
        w_list.append(th.exp(target ** 2 / 1e-2))

    for j in range(CRITIC_N_ITERATIONS):
        mse = 0
        for y_perturbed, target, w in zip(y_list, target_list, w_list):
            mse += w * critic_mse(y_bar, y_perturbed, target)
        mse /= N_PERTURBATIONS
        critic_optim.zero_grad()
        mse.backward()
        critic_optim.step()
#     hist[-1]['critic_state_dict'] = copy.deepcopy(critic.state_dict())
    
    c.train()
    critic.eval()
    c_params = copy.deepcopy(tuple(c.parameters()))
    for j in range(CLASSIFIER_N_ITERATIONS):
        y_bar = c(x)
        objective = -th.mean(critic_forward(y_bar))
        c_optim.zero_grad()
        objective.backward()
        c_optim.step()
        if any(float(th.max(th.abs(p - q))) > STD for p, q in zip(c_params, c.parameters())):
            break
    
    if (i + 1) % 1 == 0:
        f1 = L(c(x))
        print('[iteration %d]mse: %f; f1: %f' % (i + 1, mse, f1))